In [7]:
import cv2
from pathlib import Path
import os

In [8]:
#------paths---------#
images_path = Path("/home/lero/idrive/cmac/DDMAP/Image_analysis/Code/cryst_amorphous_model/images")
labels_path = Path("/home/lero/idrive/cmac/DDMAP/Image_analysis/Code/cryst_amorphous_model/labels")

output_images = Path("/home/lero/idrive/cmac/DDMAP/Image_analysis/Code/cryst_amorphous_model/patches/images")
output_labels = Path("/home/lero/idrive/cmac/DDMAP/Image_analysis/Code/cryst_amorphous_model/patches/labels")

output_images.mkdir(parents=True, exist_ok=True)
output_labels.mkdir(parents=True, exist_ok=True)

In [ ]:
#----process images------#
image_files = sorted(list(images_path.glob("*.png")))

for img_path in image_files:
    label_path=labels_path / f"{img_path.stem}.txt"
    if not label_path.exists():
        print(f"no label path found for f{img_path.name}, skipping")
        continue

    img = cv2.imread(str(img_path))
    if img is None:
        print(f"couldnt find image f{img_path}, skipping")
        continue

    h, w, _ = img.shape

    #read yolo format labels
    with open(label_path, "r") as f:
        lines=f.readlines()

    for i, line in enumerate(lines):
        parts = line.strip().split()
        if len(parts) !=5:
            continue

        cls_id, x_c, y_c, bw, bh = map(float, parts)
        #conversion to normalised coordinates
        x_c, y_c, bw, bh = x_c * w, y_c * h, bw * w, bh * h

        x1 = int(x_c - bw / 2)
        y1 = int(y_c - bh / 2)
        x2 = int(x_c + bw / 2)
        y2 = int(y_c + bh / 2)

        # Ensure coordinates are within image bounds
        x1, y1 = max(0, x1), max(0, y1)
        x2, y2 = min(w, x2), min(h, y2)

        # Crop the well
        crop = img[y1:y2, x1:x2]

        # Save cropped image
        crop_name = f"{img_path.stem}_well_{i:02d}.png"
        crop_path = output_images / crop_name
        cv2.imwrite(str(crop_path), crop)


        # Create corresponding label file (the whole crop = one object)
        label_out = output_labels / f"{crop_name.replace('.png', '.txt')}"
        with open(label_out, "w") as lf:
            lf.write(f"{int(cls_id)} 0.5 0.5 1.0 1.0\n")

    print(f"✅ Cropped {len(lines)} wells from {img_path.name}")

print("🎯 All well crops and labels created successfully!")

✅ Cropped 96 wells from 20240823_161022.png
✅ Cropped 96 wells from 20240823_161106.png
✅ Cropped 96 wells from 20240823_161147.png
✅ Cropped 96 wells from 20240828_104734.png
✅ Cropped 96 wells from 20240829_133137.png
✅ Cropped 96 wells from 20240830_103104.png
✅ Cropped 96 wells from 20240902_121458.png
✅ Cropped 96 wells from 20240907_042114.png
✅ Cropped 96 wells from 20240910_151535.png
✅ Cropped 95 wells from 20240912_113530.png
✅ Cropped 96 wells from 20240912_134010.png
✅ Cropped 96 wells from 20240912_134051.png
✅ Cropped 97 wells from 20240913_161403.png
✅ Cropped 96 wells from 20240917_150020.png
✅ Cropped 96 wells from 20241005_060027.png
✅ Cropped 96 wells from 20241007_131206.png
✅ Cropped 96 wells from 20241008_140227.png
✅ Cropped 96 wells from 20241008_140309.png
✅ Cropped 96 wells from 20241009_100529.png
✅ Cropped 96 wells from 20241009_104723.png
✅ Cropped 96 wells from 20241009_104804.png
✅ Cropped 96 wells from 20241010_114556.png
✅ Cropped 96 wells from 20241012